In [1]:
import pandas as pd

In [174]:
#metadata_path = "~Desktop/gitrepos/ncov/data/gisaid_metadata.tsv"
#metadata_df = pd.read_csv(metadata_path, delimiter="\t")
metadata_df = pd.read_csv("/Users/ablack/Desktop/gitrepos/ncov/data/gisaid_metadata.tsv", delimiter = "\t")

/Users/ablack/opt/miniconda3/envs/humboldt-analysis/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [17]:
CA_counties_with_seqs = ["Alameda County","Butte County", "Contra Costa County", "Del Norte County", "Fresno County","Humboldt County",
                        "Imperial County", "Kings County", "Lake County", "Los Angeles County", "Madera County", "Marin County", "Mariposa County", "Mendocino County",
                        "Merced County", "Mono County", "Monterey County", "Napa County", "Orange County", "Riverside County", "Sacramento County",
                        "San Bernardino County", "San Diego County", "San Francisco County", "San Joaquin County", "San Luis Obispo County", "Santa Clara County",
                        "Shasta County", "Solano County","Stanislaus County", "Tulare County", "Tuolumne County", "Ventura County"]

Here I need to make a new column to hold the spatial data that I will use for my discrete trait analysis. Notably, I don't have location information at the county level for most locations outside of CA. So, what I want to do is assign location as County for anything within CA. For anything outside of CA, I'll specify the location just at the regional level. This reduces the sheer number of demes I need to manage, and also ensures I have a record for every sequence.
Also, since I'm using this phylogeo analysis to look at introductions to Humboldt, but not looking at source sink dynamics, this feels reasonable.

In [175]:
#first step, fill info in for every record.
metadata_df['location'] = metadata_df['location'].fillna("?")
metadata_df.loc[metadata_df["location"]=="?", "adjusted_loc"] = metadata_df["region"]
metadata_df.loc[metadata_df["location"]!="?", "adjusted_loc"] = metadata_df["location"]

In [177]:
for i in metadata_df.iterrows():
    row = i[1]
    if row["adjusted_loc"] not in CA_counties_with_seqs and row["adjusted_loc"]!= row["region"]:
        metadata_df.at[i[0],"adjusted_loc"] = metadata_df.at[i[0],"region"]

In [181]:
#then drop the old location column, rename the adjusted_loc column and write to file.
adjusted_metadata_df = metadata_df.drop(columns=["location"])
adjusted_metadata_df_renamed = adjusted_metadata_df.rename({"adjusted_loc":"location"}, axis="columns")
adjusted_metadata_df_renamed.to_csv("/Users/ablack/Desktop/gitrepos/ncov/data/gisaid_metadata_location_adjusted.tsv", sep = "\t")